In [2]:
import pandas as pd
taxi_mar = pd.read_parquet('yellow_tripdata_2024-03.parquet')
taxi_apr = pd.read_parquet('yellow_tripdata_2024-04.parquet')
taxi_combined = pd.concat([taxi_mar, taxi_apr], ignore_index=True)
taxi_combined.to_csv('yellow_tripdata.csv', index=False)

In [3]:
taxi_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7096917 entries, 0 to 7096916
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           